<a href="https://colab.research.google.com/github/LevonaHayoun/3d_Unet_Segmentation/blob/main/resize%2Bstrech_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Resize images

In [ ]:
import nibabel as nib
import numpy as np
import os
from nibabel.processing import resample_to_output

def resize_nifti_image(input_image_path, output_image_path, target_shape):

    img = nib.load(input_image_path)

    original_shape = img.shape
    original_zooms = img.header.get_zooms()

    # Calculate the new voxel sizes
    new_zooms = (
        original_shape[0] * original_zooms[0] / target_shape[0],
        original_shape[1] * original_zooms[1] / target_shape[1],
        original_shape[2] * original_zooms[2] / target_shape[2],
    )

    resampled_img = resample_to_output(img, voxel_sizes=new_zooms)

    nib.save(resampled_img, output_image_path)

def process_folder(input_folder, output_folder, target_shape):

    # Iterate over all cases
    for filename in os.listdir(input_folder):
        if filename.endswith('.nii.gz'):
            input_image_path = os.path.join(input_folder, filename)
            output_image_path = os.path.join(output_folder, filename)

            # Resize the NIFTI image
            resize_nifti_image(input_image_path, output_image_path, target_shape)
            print(f'Resampled {filename} and saved to {output_image_path}')

# Define as needed
input_folder = r'C:\Users\D_Learning\Desktop\MISScnn\Case for nifty\please\imagesnew'
output_folder = r'C:\Users\D_Learning\Desktop\MISScnn\Case for nifty\please\imagesnew\resized'
target_shape = (239, 239, 154)

process_folder(input_folder, output_folder, target_shape)


#Strech images

In [ ]:
import os
import nibabel as nib
import pydicom

def dicom_grayscale_normalization(img3d, w_center, w_width):
    mask_minimum = img3d > (w_center - w_width / 2)
    mask_max = img3d < (w_center + w_width / 2)
    max_values = (~mask_max) * 3500

    nimg = w_width
    nimg -= w_center + w_width / 2 - img3d
    nimg *= 3500 / w_width

    nimg = nimg * mask_minimum
    nimg = nimg * mask_max
    nimg = nimg + max_values

    return nimg

def get_windowing_params(dicom_folder):
    slices = []
    for dicom_file in os.listdir(dicom_folder):
        if dicom_file.endswith('.dcm'):
            dicom_path = os.path.join(dicom_folder, dicom_file)
            dicom_data = pydicom.dcmread(dicom_path)
            slices.append(dicom_data)

    w_center = 0
    w_width = 0
    for i in slices:
        if i[0x0028, 0x1050].value != 128:
            w_center = i[0x0028, 0x1050].value
            break

    for i in slices:
        if i[0x0028, 0x1051].value != 255:
            w_width = i[0x0028, 0x1051].value
            break

    return w_center, w_width

def process_nifti_files(input_dir, output_dir, dicom_dir):

    # Iterate over all cases
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.nii.gz'):
            nifti_file = os.path.join(input_dir, file_name)
            nifti_img = nib.load(nifti_file)
            img3d = nifti_img.get_fdata()

            case_number = file_name.split('_')[-1].split('.')[0]

            # Find the corresponding DICOM folder
            dicom_folder = os.path.join(dicom_dir, f"case_{case_number}", 'DICOM')
            if os.path.exists(dicom_folder):
                w_center, w_width = get_windowing_params(dicom_folder)

                # Stretch the intensity values
                stretched_img = dicom_grayscale_normalization(img3d, w_center, w_width)

                output_file = os.path.join(output_dir, file_name)
                stretched_nifti_img = nib.Nifti1Image(stretched_img, nifti_img.affine, nifti_img.header)
                nib.save(stretched_nifti_img, output_file)
                print(f"Processed and saved: {output_file}")
            else:
                print(f"No corresponding DICOM folder found for {file_name}")

# Define as needed
input_dir = r'C:\Users\D_Learning\Desktop\MISScnn\Case for nifty\please\imagesnew\resized'
output_dir = r'C:\Users\D_Learning\Desktop\MISScnn\Case for nifty\please\imagesnew\stretched'
dicom_dir = r'C:\Users\D_Learning\Desktop\MISScnn\Case for nifty'

process_nifti_files(input_dir, output_dir, dicom_dir)